# Plot results
Assumes you ran the bash scripts already.



In [ ]:
%config Completer.use_jedi = False

import pandas as pd 
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid", font_scale=1)

# Plot 1 (three settings)

## load standard setting

In [ ]:
directory = 'standard_setting_tanh_15_6_reg1_smooth05'


Hopper = glob.glob(f'RESULTS/coupledflow/Hopper-v2/**/{directory}/**/*progress.txt',recursive=True)
Cheetah = glob.glob(f'RESULTS/coupledflow/HalfCheetah-v2/**/{directory}/**/*progress.txt',recursive=True)
Ant = glob.glob(f'RESULTS/coupledflow/Ant-v2/**/{directory}/**/*progress.txt',recursive=True)
Walker = glob.glob(f'RESULTS/coupledflow/Walker2d-v2/**/{directory}/**/*progress.txt',recursive=True)
Humanoid = glob.glob(f'RESULTS/coupledflow/Humanoid-v2/**/{directory}/**/*progress.txt',recursive=True)
our_runs = Hopper + Ant + Walker + Cheetah + Humanoid

print(len(our_runs), len(Hopper), len(Ant), len(Walker), len(Cheetah), len(Humanoid)) # 25, 5, 5, 5, 5.

In [ ]:
#load all into df_top with file name being column

df_top = pd.DataFrame()
total = 320000 #total timesteps
every = 4e3 # evaled every

df_top['TotalEnvInteracts'] = np.arange(1,total//every+1)*every

In [ ]:
for file_name in our_runs:
    df_temp = pd.read_csv(file_name, delimiter='\t', on_bad_lines='skip')
    if 'AverageTestEpRet' in df_temp:   
        print(len(df_temp), end=' ')
        if len(df_temp) < 80: continue #for plotting while a seed is still running
        #final underscore is for the \b in pattern matching
        df_top[file_name+ '_algo=CFIL (ours)_'] =  df_temp['AverageTestEpRet']
    else:
        print(file_name)
#out should be 80 80 80 ....

## load the two other settings (state-only and state-next-state)

In [ ]:
#load df_bootom: will include other settings state and state next


# try to view other

directory = 'state_only_tanh_15_6_reg1_smooth05'

directory2 = 'state_next_state_tanh_15_6_reg1_smooth05'


Hopper = glob.glob(f'RESULTS/coupledflow/Hopper-v2/**/{directory}/**/*progress.txt',recursive=True)
Cheetah = glob.glob(f'RESULTS/coupledflow/HalfCheetah-v2/**/{directory}/**/*progress.txt',recursive=True)
Ant = glob.glob(f'RESULTS/coupledflow/Ant-v2/**/{directory}/**/*progress.txt',recursive=True)
Walker = glob.glob(f'RESULTS/coupledflow/Walker2d-v2/**/{directory}/**/*progress.txt',recursive=True)
Humanoid = glob.glob(f'RESULTS/coupledflow/Humanoid-v2/**/{directory}/**/*progress.txt',recursive=True)

Hopper += glob.glob(f'RESULTS/coupledflow/Hopper-v2/**/{directory2}/**/*progress.txt',recursive=True)
Cheetah += glob.glob(f'RESULTS/coupledflow/HalfCheetah-v2/**/{directory2}/**/*progress.txt',recursive=True)
Ant += glob.glob(f'RESULTS/coupledflow/Ant-v2/**/{directory2}/**/*progress.txt',recursive=True)
Walker += glob.glob(f'RESULTS/coupledflow/Walker2d-v2/**/{directory2}/**/*progress.txt',recursive=True)
Humanoid += glob.glob(f'RESULTS/coupledflow/Humanoid-v2/**/{directory2}/**/*progress.txt',recursive=True)

our_runs = Hopper + Ant + Walker + Cheetah + Humanoid

print(len(our_runs), len(Hopper), len(Ant), len(Walker), len(Cheetah), len(Humanoid)) # 50, 10, 10, 10, 10, 10

In [ ]:
import pandas as pd
df_bottom = pd.DataFrame()
total = 320000 #total timesteps
every = 4e3 # evaled every

df_bottom['TotalEnvInteracts'] = np.arange(1,total//every+1)*every


In [ ]:
for file_name in our_runs:
    df_temp = pd.read_csv(file_name, delimiter='\t', on_bad_lines='skip') 
    if 'AverageTestEpRet' in df_temp:   
        print(len(df_temp),end=' ')
        if len(df_temp) < 80: #for plotting while a seed is still running
            continue
        df_bottom[file_name+ '_algo=CFIL (ours)_'] =  df_temp['AverageTestEpRet']
    else:
        print(file_name)

In [ ]:
expert_levels = {'HalfCheetah-v2':4463,'Walker2d-v2':6717,'Ant-v2':4228, 'Hopper-v2':3571, 'Humanoid-v2':5188}


# first for are from vd second two are from spinningup

envs = ['HalfCheetah-v2','Walker2d-v2','Ant-v2', 'Hopper-v2', 'Humanoid-v2']
algos = ['CFIL', 'expert']

algos_bottom = ['CFIL', 'expert']

colors = ["m","g","b","y","m","w"]

In [ ]:
#create plot for each env
fig, axs = plt.subplots(2,len(envs), figsize=(16,7))

#plot top

xaxis = range(0,int(total),int(every))#range(len(d))

for i, env in enumerate(envs):
    # create subplot
    axs[0,i].set_title(env)
    for j, alg in enumerate(algos):
        if alg == 'expert':#
            axs[0,i].hlines(expert_levels[env],xmin=0,xmax=int(total)-int(every), linestyle='--', label=alg if i == 0 else "") # avoids duplicates in legend
            continue
        d = df_top.filter(regex=fr'TotalEnvInteracts|{env}')
        index = d['TotalEnvInteracts']
        d = d.filter(regex=fr'algo={alg}')
        
        if alg == 'CFIL': alg = 'CFIL (ours)'
            
        if alg == 'bc': #special handling of bc which is offline
            avg = d.iloc[-1].mean()
            std = d.iloc[-1].std()
            axs[0,i].hlines(avg,xmin=0,xmax=int(total)-int(every),color=colors[j],label=alg if i == 0 else "") # avoids duplicates in legend
            axs[0,i].fill_between(xaxis, avg-std, avg+std,alpha=0.3,color=colors[j])
            continue
        
        avg = d.mean(axis=1)
        std = d.std(axis=1) #/2 maybe show half a standard deviation
        
        #xaxis = range(0,min(int(total), int(every)*len(d)),int(every))#range(len(d))
        
        axs[0,i].plot(xaxis,avg,color=colors[j],label=alg if i == 0 else "") # avoids duplicates in legend
        axs[0,i].fill_between(xaxis, avg-std, avg+std,alpha=0.3,color=colors[j])

        
        
#plot bottom:
        

xaxis = range(0,int(total),int(every))#range(len(d))

for i, env in enumerate(envs):
    # create subplot
    axs[1,i].set_title(env)
    for j, alg in enumerate(algos_bottom):
        if alg == 'expert':#
            axs[1,i].hlines(expert_levels[env],xmin=0,xmax=int(total)-int(every), linestyle='--', label=alg if i == 0 else "") # avoids duplicates in legend
            continue
        d = df_bottom.filter(regex=fr'TotalEnvInteracts|{env}')
        index = d['TotalEnvInteracts']
        d = d.filter(regex=fr'{alg}')
            
        if alg == 'bc': #special handling of bc which is offline
            avg = d.iloc[-1].mean()
            std = d.iloc[-1].std()
            axs[1,i].hlines(avg,xmin=0,xmax=int(total)-int(every),color=colors[j],label=alg if i == 0 else "") # avoids duplicates in legend
            axs[1,i].fill_between(xaxis, avg-std, avg+std,alpha=0.3,color=colors[j])
            continue
        
        if alg == 'CFIL': #plot state and next state separately
            d1 = d.filter(regex='state_next_state')
            d2 = d.filter(regex='state_only')

            avg = d1.mean(axis=1)
            std = d1.std(axis=1)
            
            axs[1,i].plot(xaxis,avg,color='m',label='CFIL_state_next_state')
            axs[1,i].fill_between(xaxis, avg-std, avg+std,alpha=0.3,color='m')
                          
            avg = d2.mean(axis=1)
            std = d2.std(axis=1)
            
            axs[1,i].plot(xaxis,avg,color='g',label='CFIL_state')
            axs[1,i].fill_between(xaxis, avg-std, avg+std,alpha=0.3,color='g')
            continue

        
        
#change tick...
for ax in axs.flatten():
    ax.tick_params(size=1,labelsize=9)
    ax.ticklabel_format(axis='x',style="sci", scilimits=(0,0))
    ax.set_xlabel('Env int',fontsize=10)

#only left most will have
axs[0,0].set_ylabel('Env Return', fontsize=10)
axs[1,0].set_ylabel('Env Return', fontsize=10)
   
#fig.legend(loc=(0.1,0.15),framealpha=0.5)
axs[0,0].legend(loc='lower center',framealpha=0.8)
axs[1,0].legend(loc='lower right',framealpha=0.8, prop={'size': 10})

plt.subplots_adjust(hspace=0.3) #was not saving simply because i did this line after the savefig
 
#fig.savefig('final_plot1.png')



#change ticks

# Plot 2 (subsampled results)

In [ ]:
directory = "all_subsampled"


Hopper = glob.glob(f'RESULTS/coupledflow/Hopper-v2/**/{directory}/**/*progress.txt',recursive=True)
Cheetah = glob.glob(f'RESULTS/coupledflow/HalfCheetah-v2/**/{directory}/**/*progress.txt',recursive=True)
Ant = glob.glob(f'RESULTS/coupledflow/Ant-v2/**/{directory}/**/*progress.txt',recursive=True)
Walker = glob.glob(f'RESULTS/coupledflow/Walker2d-v2/**/{directory}/**/*progress.txt',recursive=True)
Humanoid = glob.glob(f'RESULTS/coupledflow/Humanoid-v2/**/{directory}/**/*progress.txt',recursive=True)
our_runs = Hopper + Ant + Walker + Cheetah + Humanoid

runs = Hopper + Humanoid + Cheetah + Ant + Walker

print(len(Hopper),len(Humanoid),len(Walker), len(Ant),len(Cheetah)) # 20 20 20 20 20

In [ ]:
df2 = pd.DataFrame()
total = 320000 #total timesteps 
every = 4e3 # evaled every

df2['TotalEnvInteracts'] = np.arange(1,total//every+1)*every

In [ ]:
for file_name in runs:
    df_temp = pd.read_csv(file_name, delimiter='\t', on_bad_lines='skip')
    if 'AverageTestEpRet' in df_temp:   
        print(len(df_temp),end=' ')
        if len(df_temp) < 80: #for plotting while a seed is still running
            continue
        df2[file_name+ '_algo=CFIL (ours)_'] =  df_temp['AverageTestEpRet']
    else:
        print(file_name)

In [ ]:

expert_levels = {'HalfCheetah-v2':4463,'Walker2d-v2':6717,'Ant-v2':4228, 'Hopper-v2':3571, 'Humanoid-v2':5188}


settings = ['subsample10','subsample20', 'subsample50', 'subsample100']



envs = ['HalfCheetah-v2','Walker2d-v2','Ant-v2', 'Hopper-v2', 'Humanoid-v2']
algos = ['CFIL','expert']


colors = ["g","r","b","y","m","w"]

fig, axs = plt.subplots(len(envs),len(settings), figsize=(len(envs)*2,len(settings)*2))

if len(envs) == 1:
    axs = [axs]

xaxis = range(0,int(total),int(every))#range(len(d))


for i, env in enumerate(envs):
    # create subplot
    
    for j, alg in enumerate(algos):
        for k, setting in enumerate(settings):
            axs[0,k].set_title(setting) #only top will have title
            axs[i,0].set_ylabel(env) # left 
            if alg == 'expert':#
                axs[i,k].hlines(expert_levels[env],xmin=0,xmax=int(total)-int(every), linestyle='--', label=alg if i + k == 0 else "") # avoids duplicates in legend
                continue
            d = df2.filter(regex=fr'TotalEnvInteracts|{env}')
            index = d['TotalEnvInteracts']
            d = d.filter(regex=fr'{alg}') #underscore avoids subsample100 matching subsample10
            d = d.filter(regex=fr'{setting}_|{setting}\.') #avoids subsample100 matching subsample10
            #if alg != 'expert': alg = 'CFIL_'+alg

            print(env,setting,len(d.columns))
            avg = d.mean(axis=1)
            std = d.std(axis=1) #/2 maybe show half a standard deviation

            #xaxis = range(0,min(int(total), int(every)*len(d)),int(every))#range(len(d))
            
            axs[i,k].plot(xaxis,avg,color=colors[j],label=alg.upper() if i + k == 0 else "") # avoids duplicates in legend
            axs[i,k].fill_between(xaxis, avg-std, avg+std,alpha=0.3,color=colors[j])
    
for i, a in enumerate(axs):
    for j, b in enumerate(a):
        b.tick_params(size=1,labelsize=9)
        b.ticklabel_format(axis='x',style="sci", scilimits=(0,0))
        
        if i !=len(envs)-1:
            b.set_xticklabels([])
        if j!=0:
            b.set_yticklabels([])
#change tick...
#for ax in axs.flatten():
    #ax.tick_params(size=1,labelsize=9)
    #ax.ticklabel_format(axis='x',style="sci", scilimits=(0,0))
    #ax.set_xlabel('Env int',fontsize=10)
        
axs[0,0].legend(prop={'size': 13},framealpha=0.5)

plt.subplots_adjust(wspace=0.05, hspace=0.05)


#align all graphs to the left most one...
for i, a in enumerate(axs):
    for j, b in enumerate(a):            
            b.set_ylim(a[0].get_ylim())


#loc=(0.1,0.15)
#legend size ,prop={'size': 10}

#fig.savefig('final_plot2.png')

#change ticks